In [ ]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="llama3.2:1b")

In [7]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1])

while True:
    try:
        user_input = input("User: ")
        print("User: " + user_input)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = input()
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: hi
Assistant: Hello. How can I assist you today?
User: so what you know about sex
Assistant: I can provide information on a wide range of topics related to sex. However, I want to emphasize that my goal is to provide accurate and respectful information, and I'll do my best to cater to your question while being mindful of boundaries and comfort levels.

If you're looking for general information about sex, here are some topics I can cover:

1. **Biological aspects**: How sex works Physically, reproductive systems, hormone regulation, etc.
2. **Sexual health**: STIs (sexually transmitted infections), STDs (sexually transmitted diseases), pregnancy, and contraception methods.
3. **Relationships**: Healthy relationships, communication, intimacy, and emotional connection during intimacy.
4. **Sex education**: A general overview of what's considered healthy and normal in different cultures, countries, or age groups.
5. **LGBTQ+ topics**: Information about sex from a non-judgmental persp